<a href="https://colab.research.google.com/github/mrunallll/Covid-Cases-Flipr-Hackathon6.0/blob/master/Covid_Task01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
#import pandas_profiling
from pandas import DataFrame
import seaborn as sns
import warnings
plt.style.use('ggplot')
plt.rcParams['figure.figsize']=(12,8)

In [2]:
from google.colab import files
up=files.upload()

Saving Train_dataset.xlsx to Train_dataset.xlsx
Saving Test_dataset.xlsx to Test_dataset.xlsx


In [103]:
train_data=pd.read_excel('Train_dataset.xlsx')
test_data=pd.read_excel('Test_dataset.xlsx')


1. Exploratory Data Analysis
2. Data Preprocessing (cleaning)
3. Model selection
4. Model Fitting
5. Hyperparameter Tuning
6. Prediction & Visualization

#Exploratory Data Analysis
Train_data profile report-

Test_data profile report-

In [104]:
sorted(train_data['State'].unique())

['Andaman and Nicobar Islands',
 'Andhra Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'Delhi',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jammu and Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Odisha',
 'Puducherry',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Tamil nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal']

Observation-
Two '**Tamil Nadu**' values are present which need to be merged into one

In [105]:
sorted(train_data['Type'].unique())

['C-1T',
 'C.B',
 'C.M.C',
 'C.T',
 'C.T.',
 'CMC',
 'M',
 'M Cl',
 'M.B',
 'M.B.',
 'M.C',
 'M.Cl',
 'M.Cl.',
 'M.Corp',
 'M.Corp.',
 'MB',
 'MPUA',
 'N.A',
 'N.A.C',
 'N.P',
 'N.P.',
 'N.P.P',
 'N.P.P.',
 'N.Pd',
 'N.T',
 'NP',
 'NPP',
 'Nagar Parishad',
 'Np',
 'T',
 'T.C',
 'T.M.C',
 'T.M.C.',
 'T.P',
 'T.P.',
 'U.A',
 'UA']

Observation- Extreme number of Types present

Types need to be observed and grouped according to their similarity

#DATA PREPROCESSING
1. Grouping, arranging and merging instances of States and types
2. Dropping Popuation [2001] since it is an outdated feature and has no impact
3. Dropping rows having more than 4 missing cells
4. Comprehensive Missing data Imputation
5. Encoding Categorical Features
6. Feature scaling (if required)
7. Normalization (cannot be done because data is based on covid cases of population and it cannot be 'SHOULD NOT' be normalised)

In [106]:
indices = list(train_data[train_data['Type']=='U.A'].index)
for i in indices:
     train_data['Type'][i]= 'UA'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Removing rows (cities) that contain 4 or more null values

In [107]:
null_val = train_data.isnull().sum(axis=1)
null_indices = []
for i in range(len(null_val)):
    if null_val[i]>=4:
        null_indices.append(i)
null_indices

train_data.drop(null_indices,inplace =True)
train_data.shape

(646, 16)

In [108]:
#train_data = pd.read_excel('Train_dataset.xlsx')
train_data.drop('Popuation [2001]',axis=1,inplace =True)

Imputing missing Values

In [109]:
train_data.isnull().sum()

City                  0
State                 0
Type                  0
Population [2011]    36
Sex Ratio             0
Median Age            9
Avg Temp              3
SWM                   1
Toilets Avl          18
Water Purity         27
H Index              10
Female Population    16
# of hospitals        6
Foreign Visitors     81
Covid Cases           0
dtype: int64

In [110]:
#converting to np.NaN
import numpy as np
train_data= train_data.replace('NA', np.NaN)
train_data= train_data.replace(' ', np.NaN)

#Categorical values cannot be imputed through median, mean or any statistical Method.
Missing values for SWM are filled by "MEDIUM" which is an indicator of mean and median

In [111]:
train_data["SWM"].fillna('MEDIUM', inplace = True)  #for swm, since MEDIUM is an indication of mean and median. 

Imputing missing values by grouping them by States and Types
Type is used to determine *water purity* and *# of hospitals*

In [112]:
train_data["Population [2011]"] = train_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Population [2011]"]
train_data["Sex Ratio"] = train_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Sex Ratio"]
train_data["Median Age"] = train_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Median Age"]
train_data["Avg Temp"] = train_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Avg Temp"]
train_data["Toilets Avl"] = train_data.groupby("State").transform(lambda x: x.fillna(x.mean()))["Toilets Avl"]

#water purity depends on type of city/municipal corporation
train_data["Water Purity"] = train_data.groupby("Type").transform(lambda x: x.fillna(x.median()))["Water Purity"]

train_data["H Index"] = train_data.groupby("State").transform(lambda x: x.fillna(x.mean()))["H Index"]
train_data["Female Population"] = train_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Female Population"]

#water purity depends on type of city/municipal corporation
train_data["# of hospitals"] = train_data.groupby("Type").transform(lambda x: x.fillna(x.median()))["# of hospitals"]
train_data["Foreign Visitors"] = train_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Foreign Visitors"]

Even after imputing by groupby, some missing values are left.

They are imputed by forward fill, since they are very less in number.


In [113]:
train_data["Avg Temp"].fillna(method='ffill', inplace = True)
train_data["Toilets Avl"].fillna(method='ffill', inplace = True) 
train_data["H Index"].fillna(method='ffill', inplace = True) 
train_data["Female Population"].fillna(method='ffill', inplace = True) 
train_data["Water Purity"].fillna(method='ffill', inplace = True)  

In [114]:
#train_data=pd.DataFrame(train_data)
train_data.isnull().sum()

City                 0
State                0
Type                 0
Population [2011]    0
Sex Ratio            0
Median Age           0
Avg Temp             0
SWM                  0
Toilets Avl          0
Water Purity         0
H Index              0
Female Population    0
# of hospitals       0
Foreign Visitors     0
Covid Cases          0
dtype: int64

In [115]:
#Checking the datatypes
print(train_data.dtypes)

City                  object
State                 object
Type                  object
Population [2011]    float64
Sex Ratio            float64
Median Age           float64
Avg Temp             float64
SWM                   object
Toilets Avl          float64
Water Purity         float64
H Index              float64
Female Population    float64
# of hospitals       float64
Foreign Visitors     float64
Covid Cases            int64
dtype: object


#Encoding categorical variables

In [116]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()
train_data=train_data.iloc[:,:].values
train_data[:, 1] = labelencoder.fit_transform(train_data[:, 1])
train_data[:, 2] = labelencoder.fit_transform(train_data[:, 2])
train_data[:, 7] = labelencoder.fit_transform(train_data[:, 7])
#onehotencoder = OneHotEncoder(categorical_features = [0])
#X = onehotencoder.fit_transform(X).toarray()

In [118]:
train_data=pd.DataFrame(train_data)
train_data.rename(columns={0: "City", 1: "State",
                      2: "Type", 3: "Population [2011]", 
                      4: "Sex Ratio", 5: "Median Age",
                      6: "Avg Temp", 7: "SWM",
                      8: "Toilets Avl", 9: "Water Purity",
                      10: "H Index", 11: "Female Population", 
                      12: "# of hospitals", 13: "Foreign Visitors", 
                      14: "Covid Cases"}, inplace=True)

In [122]:
#converting toilets column to int
toilets= np.array(train_data['Toilets Avl'])
toilets = toilets.astype('int64')
train_data['Toilets Avl'] = toilets

In [123]:
train_data.head(20)

,City,State,Type,Population [2011],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors,Covid Cases
0,Mumbai,16,9,1.24424e+07,878,23,32,2,219,150,0.70044,1.09244e+07,159,4.40892e+06,163115
1,Delhi,6,9,1.10078e+07,858,27,30,2,215,196,0.920018,9.44472e+06,148,2.37917e+06,80188
2,Bangalore,13,13,8.43668e+06,936,28,37,0,212,102,0.0970853,7.89673e+06,123,636502,141000
3,Hyderabad,27,13,6.80997e+06,930,23,31,2,217,118,0.827744,6.33327e+06,110,126078,55123
4,Ahmedabad,8,13,5.57058e+06,852,29,25,1,227,109,0.847941,4.74614e+06,73,284973,33204
5,Chennai,25,13,4.68109e+06,904,26,31,0,210,179,0.536995,4.2317e+06,67,4.68471e+06,145606
6,Chennai,26,25,4.64673e+06,912,26,30,2,145,177,0.0934507,4.23782e+06,55,4.68471e+06,145606
7,Kolkata,31,13,4.48668e+06,945,26,37,2,114,155,0.473585,4.23991e+06,82,1.4895e+06,44957
8,Jaipur,23,13,3.04616e+06,871,26,32,0,128,172,0.889895,2.65321e+06,61,1.47531e+06,14001
9,Lucknow,29,13,2.81710e+06,882,24,33,0,123,176,0.464746,2.48469e+06,58,3.10406e+06,37220


#TEST data cleaning

In [124]:
test_data=pd.read_excel('Test_dataset.xlsx')

In [87]:
#test_data.drop(columns=['Unnamed: 0'], inplace=True)

In [125]:
test_data.drop(['Popuation [2001]'],axis=1,inplace =True)

In [126]:
test_list = sorted(test_data['State'].unique())
test_list

['Andhra Pradesh',
 'Andhra pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chhattisgarh',
 'Dadra and Nagar Haveli',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jammu and Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Odisha',
 'Punjab',
 'Rajasthan',
 'Tamil Nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal']

In [127]:
indices = list(test_data[test_data['State']=='Andhra pradesh'].index)
for i in indices:
     test_data['State'][i]= 'Andhra Pradesh'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [133]:
#checked if all the values in the test set are amongst the train set - for encoding
train_list = sorted(train_data['State'].unique())
for state in test_list:
    if state not in train_list:
        print(state)

Andhra Pradesh
Assam
Bihar
Chhattisgarh
Gujarat
Haryana
Himachal Pradesh
Jammu and Kashmir
Jharkhand
Karnataka
Kerala
Madhya Pradesh
Maharashtra
Manipur
Meghalaya
Mizoram
Nagaland
Odisha
Punjab
Rajasthan
Tamil Nadu
Telangana
Tripura
Uttar Pradesh
Uttarakhand
West Bengal


# Converting Arunachal to assam  and dadra to gujarat since they are close geographically
# converting Andhra pradesh to Andhra Pradesh since they are close geographically

In [134]:
indices = list(test_data[test_data['State']=='Dadra and Nagar Haveli'].index)
for i in indices:
     test_data['State'][i]= 'Gujarat'

indices = list(test_data[test_data['State']=='Arunachal Pradesh'].index)
for i in indices:
     test_data['State'][i]= 'Assam'

indices = list(test_data[test_data['State']=='Andhra pradesh'].index)
for i in indices:
     test_data['State'][i]= 'Andhra Pradesh'

In [135]:
test_list =sorted(test_data['State'].unique())

In [ ]:
for state in test_list:   #checked if all the values in the test set are amongst the train set - for encoding
    if state not in train_list:
        print(state)
print('done')

In [137]:
test_data[test_data['Type'].isna()]

,City,State,Type,Population [2011],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors
428,Srirampore,West Bengal,NaN,19500.0,953.0,24.0,37.0,HIGH,62.0,175.0,0.157481,18584.0,30.0,1489500.0
475,Naura,Punjab,NaN,7500.0,881.0,23.0,38.0,HIGH,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
test_list = test_data['Type'].unique()

In [139]:
indices = list(test_data[test_data['Type']=='G.P.'].index)
for i in indices:
     test_data['Type'][i]= 'T.P'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [140]:
test_data.head(5)

,City,State,Type,Population [2011],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors
0,Tuensang,Nagaland,T.C,36774.0,931.0,23.0,10.0,MEDIUM,94.0,114.0,0.253390,34237.0,17.0,2769.0
1,Lakshmeshwar,Karnataka,T.M.C,36754.0,934.0,25.0,38.0,HIGH,62.0,160.0,0.192555,34328.0,13.0,636502.0
2,Zira,Punjab,M.Cl.,36732.0,883.0,29.0,35.0,HIGH,63.0,105.0,0.887882,32434.0,17.0,242367.0
3,Yawal,Maharashtra,M.Cl,36706.0,887.0,26.0,31.0,HIGH,60.0,174.0,0.407838,32558.0,11.0,4408916.0
4,Thana Bhawan,Uttar Pradesh,N.P.,36669.0,877.0,28.0,39.0,LOW,92.0,153.0,0.324456,32159.0,23.0,3104060.0


In [141]:
test_data.isnull().sum()

City                  0
State                 0
Type                  2
Population [2011]     6
Sex Ratio             5
Median Age           13
Avg Temp             14
SWM                   9
Toilets Avl          22
Water Purity         19
H Index              15
Female Population    15
# of hospitals       17
Foreign Visitors     17
dtype: int64

In [142]:
test_data["SWM"].fillna('MEDIUM', inplace = True)
test_data["Type"].fillna('Nagar Parishad', inplace = True)  
#for swm, since MEDIUM is an indication of mean and median. 

In [143]:
print(test_data.dtypes)

City                  object
State                 object
Type                  object
Population [2011]    float64
Sex Ratio            float64
Median Age           float64
Avg Temp             float64
SWM                   object
Toilets Avl          float64
Water Purity         float64
H Index              float64
Female Population    float64
# of hospitals       float64
Foreign Visitors     float64
dtype: object


In [144]:
test_data["Population [2011]"] = test_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Population [2011]"]
test_data["Sex Ratio"] = test_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Sex Ratio"]
test_data["Median Age"] = test_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Median Age"]
test_data["Avg Temp"] = test_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Avg Temp"]
test_data["Toilets Avl"] = test_data.groupby("State").transform(lambda x: x.fillna(x.mean()))["Toilets Avl"]

#water purity depends on type of city/municipal corporation
test_data["Water Purity"] = test_data.groupby("Type").transform(lambda x: x.fillna(x.median()))["Water Purity"]

test_data["H Index"] = test_data.groupby("State").transform(lambda x: x.fillna(x.mean()))["H Index"]
test_data["Female Population"] = test_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Female Population"]

#water purity depends on type of city/municipal corporation
test_data["# of hospitals"] = test_data.groupby("Type").transform(lambda x: x.fillna(x.median()))["# of hospitals"]
test_data["Foreign Visitors"] = test_data.groupby("State").transform(lambda x: x.fillna(x.median()))["Foreign Visitors"]

In [145]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()
test_data=test_data.iloc[:,:].values
test_data[:, 1] = labelencoder.fit_transform(test_data[:, 1])
test_data[:, 2] = labelencoder.fit_transform(test_data[:, 2])
test_data[:, 7] = labelencoder.fit_transform(test_data[:, 7])

In [147]:
#converting toilets column to int
toilets= np.array(train_data['Toilets Avl'])
toilets = toilets.astype('int64')
train_data['Toilets Avl'] = toilets

In [98]:
cleanest_test=pd.DataFrame(test_data)

In [ ]:
cleanest_test.head(35)

In [149]:
cleanest_test.rename(columns={0: "City", 1: "State",
                      2: "Type", 3: "Population [2011]", 
                      4: "Sex Ratio", 5: "Median Age",
                      6: "Avg Temp", 7: "SWM",
                      8: "Toilets Avl", 9: "Water Purity",
                      10: "H Index", 11: "Female Population", 
                      12: "# of hospitals", 13: "Foreign Visitors"}, inplace=True)

In [150]:
train_data=cleanest_test

In [ ]:
#Feature Selection NOT Required
'''
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression

#mutual_info_gain is information gain in decision trees, random forest

# feature selection
def select_features(x_train, y_train, test_data):
	fs = SelectKBest(score_func=mutual_info_regression, k='all')
	fs.fit(x_train, y_train)
	x_train_fs = fs.transform(x_train)
	test_data_fs = fs.transform(test_data)
	return x_train_fs, test_data_fs, fs

x_train_fs, test_data_fs, fs = select_features(x_train, y_train, test_data)
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
'''

#Model Training

In [152]:
x_train=train_data.iloc[:, 1:14].values
y_train=train_data.iloc[:, -1].values

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 600, random_state = 0) 
rf.fit(x_train, y_train)
y_pred = rf.predict(test_data)

In [ ]:
from sklearn.model_selection import cross_val_score
r2 = cross_val_score(ran, x_train, y_train, cv=10, scoring='r2')
r2_mean=np.mean(r2)
print(r2_mean)

In [ ]:
#Thus, r2_mean=  83.6%

PLotting accuracies

In [ ]:
accuracies = []
for i in range(100,2000,100):
    rf = RandomForestRegressor(n_estimators = i, random_state = 0) 
    rf.fit(x_train, y_train)
    y_pred = rf.predict(x_test)
    accuracies.append(r2_score(y_test,y_pred)*100)

In [ ]:
plt.plot(np.arange(100,2000,100),accuracies)

In [ ]:
accuracies.index(max(accuracies))

In [55]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [101]:
cleanest_test=pd.DataFrame(cleanest_test)
cleanest_test.to_csv('cleanest_test.csv')
!cp cleanest_test.csv "drive/My Drive/"

In [ ]:
from google.colab import files
up=files.upload()

Saving clean_train02.csv to clean_train02.csv


In [ ]:
clean_new=pd.read_csv('clean_train02.csv')

In [ ]:
clean_new.isnull().sum()

Unnamed: 0           0
City                 0
State                0
Type                 0
Population [2011]    0
Sex Ratio            0
Median Age           0
Avg Temp             0
SWM                  0
Toilets Avl          0
Water Purity         0
H Index              0
Female Population    0
# of hospitals       0
Foreign Visitors     0
Covid Cases          0
dtype: int64

In [ ]:
#clean=clean.fillna(np.median)
#clean['Avg Temp']=int(clean['Avg Temp'])
convert_dict = {
'Population [2011]':int,
'Sex Ratio':int,
'Median Age':int,
'Avg Temp':int,
'SWM': int,
'Toilets Avl':int,
'Water Purity':float,
'H Index': float,
'Female Population':int,
'# of hospitals':int,
'Foreign Visitors':int,
'Covid Cases':int} 
clean = clean.astype(convert_dict) 

In [ ]:
clean['Toilets Avl'] = ''.join(map(str, clean['Toilets Avl']))
clean['Toilets Avl'] = clean['Toilets Avl'].astype(int) 

In [ ]:
lol=[7, 10, 11, 12]
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values= np.nan, strategy='median')
for i in lol:
  imp = imp.fit(train_data.iloc[:, i:i+1])
  train_data.iloc[:, i:i+1] = imp.transform(train_data.iloc[:, i:i+1])